In [8]:
from algorithm.ppo import Learner
from configs.args_parser import Parameters
from configs import Factor_dictionary
from algorithm.analysis import MO_Analysis
from algorithm.utils import MO_Stats
import mo_gymnasium as mo_gym 
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from algorithm.utils import make_styled_env
from io import BytesIO
import os
import io
import gymnasium as gym

env_id = 'minecart-v0'


In [1]:
from rlmodel.vector_net import VExpert,VStyleExpert,VMLPStyleExpert

In [4]:
vexpert = VExpert(10,2,4)
vstyleexpert = VStyleExpert(10,2,4,4)

In [10]:
env_id = 'deep-sea-treasure-v0'
env = mo_gym.make(env_id)
env = mo_gym.LinearReward(env, weight=np.array([1,1]))

In [11]:
env_list = []
for _ in range(5):
    env_list.append(make_styled_env(env_id,0,[1,1]))
envs = gym.vector.SyncVectorEnv(env_list)
envs = gym.wrappers.RecordEpisodeStatistics(envs)


In [12]:
o,info = envs.reset()

In [13]:
for _ in range(5):
    a = envs.action_space.sample()
    o,r,d,t,info = envs.step(a)

In [25]:
vector_rewards = np.stack([info['vector_reward'][i] for i in range(len(info['vector_reward']))])

ValueError: all input arrays must have the same shape

In [33]:
res = []
for i in range(len(info['vector_reward'])):
    if info['vector_reward'][i] is not None:
        res.append(info['vector_reward'][i])
    else:
        res.append(info['final_info'][i]['vector_reward'])

res

[array([ 0., -1.], dtype=float32),
 array([ 0., -1.], dtype=float32),
 array([ 0., -1.], dtype=float32),
 array([ 0., -1.], dtype=float32),
 array([ 0.7, -1. ], dtype=float32)]

In [34]:
res = np.stack(res,axis=0)


In [37]:
res_th = torch.from_numpy(res).float()
res_th.mean(-1,keepdim=True)

tensor([[-0.5000],
        [-0.5000],
        [-0.5000],
        [-0.5000],
        [-0.1500]])

In [38]:
res_th.shape

torch.Size([5, 2])

In [41]:
adv = torch.randn(10,2)
weights = torch.randn(10,2)

adv * weights


In [44]:
res = np.random.randn(10,2)
1 - np.var(res,axis=0)

array([0.2563575 , 0.31160715])

In [46]:
var_res = np.var(res,axis=0)
np.any(np.isnan(var_res))

False

In [48]:
np.any(var_res == 0)

False